In [1]:
from ds_util import get_dataset
from model import MLPModel
import numpy as np
import torch
import h5py
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from scipy.stats import spearmanr, pearsonr
from tqdm import tqdm

In [2]:
predictions_savepath = '/clusterfs/nilah/oberon/datasets/cs282a/inference/maxpool_model_20231128_072156_3.h5'

In [3]:
f = h5py.File('/clusterfs/nilah/oberon/datasets/basenji/embeddings/embeddings.h5','r')

In [4]:
dset = f['embeddings']

In [5]:
trained_model = MLPModel()
trained_model.load_state_dict(torch.load('model_20231128_072156_3'))
trained_model.eval()

MLPModel(
  (layers): Sequential(
    (conv1x1): Conv1d(1536, 500, kernel_size=(1,), stride=(1,))
    (gelu1): GELU(approximate='none')
    (maxpool1): MaxPool1d(kernel_size=896, stride=896, padding=0, dilation=1, ceil_mode=False)
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (fc1): Linear(in_features=500, out_features=18, bias=True)
  )
)

In [6]:
with h5py.File(predictions_savepath,'w') as savefile:
    savefile.create_dataset(
        'single_bin',
        shape=(len(dset),1,18),
        chunks=(1,1,18),
        compression='gzip',
        compression_opts=9,
    )
    for i in tqdm(range(len(dset))):
        inputs = torch.Tensor(f['embeddings'][i]).reshape([1,896,1536])
        predictions = trained_model(inputs.transpose(1,2))
        savefile['single_bin'][i,:,:] = predictions.detach()


100%|██████████| 38171/38171 [32:37<00:00, 19.50it/s]  
